In [326]:
import mysql.connector
import pandas as pd
import glob
import os
import datetime
from datetime import datetime
import itertools

# Conexión a la base de datos
db = mysql.connector.connect(
    host="192.168.1.66",
    user="root",
    password="admin",
    database="Stock"
)

cursor = db.cursor(buffered=True)

In [327]:
ruta_base = r"\\layla\\Documentos\\STOCK\\"
ruta_carpetas_base = glob.glob(ruta_base + 'Sto*')
rutas_imagen_CCU = []
rutas_imagen_ADA = []

In [328]:
for ruta in ruta_carpetas_base:    
    ruta_ADA = os.path.join(ruta, '01- ADA')
    if os.path.exists(ruta_ADA):
            # Construye la ruta hacia la carpeta "IMAGEN" dentro de "10- ROTURAS"
            ruta_imagen_ADA = os.path.join(ruta_ADA, 'FECHAS PREVENTA')

            # Verifica si la carpeta "IMAGEN" existe
            if os.path.exists(ruta_imagen_ADA):
                rutas_imagen_ADA.append(ruta_imagen_ADA)

rutas_meses = []

for ruta_imagen in rutas_imagen_ADA:
    # Busca todas las subcarpetas dentro de la carpeta "IMAGEN"
    subcarpetas = glob.glob(os.path.join(ruta_imagen, '*/'))

    # Añade las rutas de las subcarpetas a la lista rutas_meses
    rutas_meses.extend(subcarpetas)

rutas_archivos_ADA = []

for ruta_mes in rutas_meses:
    # Lista todos los archivos en la subcarpeta
    archivos = glob.glob(os.path.join(ruta_mes, '*'))
    # Extiende la lista rutas_archivos_ADA con los elementos de archivos
    rutas_archivos_ADA.extend(archivos)


In [329]:
for ruta in ruta_carpetas_base:    
    ruta_CCU = os.path.join(ruta, '02- CCU')
    if os.path.exists(ruta_CCU):
            # Construye la ruta hacia la carpeta "IMAGEN" dentro de "10- ROTURAS"
            ruta_imagen_CCU = os.path.join(ruta_CCU, '06- FECHAS')

            # Verifica si la carpeta "IMAGEN" existe
            if os.path.exists(ruta_imagen_CCU):
                rutas_imagen_CCU.append(ruta_imagen_CCU)

rutas_meses = []

for ruta_imagen in rutas_imagen_CCU:
    # Busca todas las subcarpetas dentro de la carpeta "IMAGEN"
    subcarpetas = glob.glob(os.path.join(ruta_imagen, '*/'))

    # Añade las rutas de las subcarpetas a la lista rutas_meses
    rutas_meses.extend(subcarpetas)

rutas_archivos_CCU = []

for ruta_mes in rutas_meses:
    # Lista todos los archivos en la subcarpeta
    archivos = glob.glob(os.path.join(ruta_mes, '*'))
    rutas_archivos_CCU.append(archivos)

rutas_archivos_CCU_plano = list(itertools.chain(*rutas_archivos_CCU))

In [330]:
# Lista de nombres de pestañas a buscar
nombres_pestanas = ["DEPOSITO", "AZOPARDO", "SALON"]

datos_ADA = []

for archivo in rutas_archivos_ADA:
    # Cargar el archivo Excel
    fecha_creacion = pd.to_datetime(os.path.getctime(archivo), unit='s')
    xls = pd.ExcelFile(archivo)

    # Iterar sobre cada hoja del libro
    for hoja in xls.sheet_names:
        # Verificar si la hoja contiene alguna de las pestañas requeridas
        if any(pestaña in hoja for pestaña in nombres_pestanas):
            # Cargar datos de la hoja actual
            datos_hoja = pd.read_excel(archivo, sheet_name=hoja)

            datos_hoja['Fecha Inv.'] = fecha_creacion
            datos_hoja['Fecha Inv.'] = datos_hoja['Fecha Inv.'].dt.date

            # Agregar los datos al resultado
            datos_ADA.append(datos_hoja)

# Concatenar todos los datos recopilados
datos_ADA = pd.concat(datos_ADA, ignore_index=True)

datos_ADA = datos_ADA[['SKU','Fecha Inv.', 'Vencimiento','Unnamed: 9',  'TOTAL Bultos']]
datos_ADA.rename(columns= {'Unnamed: 9' : 'Ubicacion', 'Fecha Inv.': 'fecha_reelevamiento'}, inplace= True)
datos_ADA.dropna(inplace=True)

In [331]:
# Lista de nombres de pestañas a buscar
nombres_pestanas = ["Mayorista", "Deposito", "Barriles", "Azopardo"]

datos_CCU = []

for archivo in rutas_archivos_CCU_plano:
    # Cargar el archivo Excel
    xls = pd.ExcelFile(archivo)

    # Iterar sobre cada hoja del libro
    for hoja in xls.sheet_names:
        # Verificar si la hoja contiene alguna de las pestañas requeridas
        if any(pestaña in hoja for pestaña in nombres_pestanas):
            # Cargar datos de la hoja actual
            datos_hoja = pd.read_excel(archivo, sheet_name=hoja)

            # Agregar los datos al resultado
            datos_CCU.append(datos_hoja)

# Concatenar todos los datos recopilados
datos_CCU = pd.concat(datos_CCU, ignore_index=True)
datos_CCU = datos_CCU[['SKU','Fecha Inv.', 'Plant', 'Vencimiento', 'Total Bultos']]
datos_CCU.rename(columns= {'Fecha Inv.' : 'fecha_reelevamiento', 'Plant': 'Ubicacion', 'Total Bultos': 'TOTAL Bultos'}, inplace= True)
datos_CCU.dropna(inplace=True)


In [332]:
datos_ADA

,SKU,fecha_reelevamiento,Vencimiento,Ubicacion,TOTAL Bultos
0,25184.0,2023-05-22,2023-07-05,deposito,289.0
1,22639.0,2023-05-22,2023-07-09,deposito,12.0
2,24112.0,2023-05-22,2023-07-24,deposito,2.0
3,25232.0,2023-05-22,2023-07-28,deposito,129.0
4,20475.0,2023-05-22,2023-08-02,deposito,327.0
...,...,...,...,...,...
1021,25006.0,2023-11-14,2024-09-07,salon,52.0
1022,25113.0,2023-11-14,2024-09-16,salon,44.0
1023,25048.0,2023-11-14,2024-09-24,salon,4.0
1024,25002.0,2023-11-14,2024-09-29,salon,48.0


In [333]:
datos_CCU

,SKU,fecha_reelevamiento,Ubicacion,Vencimiento,TOTAL Bultos
1,22645.0,2023-01-06,SALON MAYORISTA,2023-02-26,80.0
2,20608.0,2023-01-06,SALON MAYORISTA,2023-02-27,1.0
3,21528.0,2023-01-06,SALON MAYORISTA,2023-03-28,11.0
4,18873.0,2023-01-06,SALON MAYORISTA,2023-04-20,3.0
5,21149.0,2023-01-06,SALON MAYORISTA,2023-05-03,11.0
...,...,...,...,...,...
5752,24927.0,2023-11-17,MAGALLANES,2024-02-05,24.0
5753,24927.0,2023-11-17,MAGALLANES,2024-03-19,42.0
5754,21122.0,2023-11-17,MAGALLANES,2024-02-26,10.0
5756,9375.0,2023-11-17,AZOPARDO,2024-07-21,4680.0


In [334]:
df_final = pd.concat([datos_ADA, datos_CCU])

In [335]:
# Mapeo de valores a estandarizar
mapeo = {
    'deposito': 'MAGALLANES',
    'MAGALLANES': 'MAGALLANES',
    'salon': 'SALON MAYORISTA',
    'SALON MAYORISTA': 'SALON MAYORISTA',
    'Azopardo': 'AZOPARDO',
    'AZOPARDO': 'AZOPARDO'
}

# Aplicar la estandarización usando la función replace
df_final['Ubicacion'] = df_final['Ubicacion'].replace(mapeo)

# Diccionario de mapeo de ubicaciones a IDs
mapeo_ubicaciones = {
    'MAGALLANES': 1,
    'SALON MAYORISTA': 2,
    'AZOPARDO': 3,
    'COUSTLAIN': 4
}

# Aplicar la sustitución usando la función replace
df_final['Ubicacion'] = df_final['Ubicacion'].replace(mapeo_ubicaciones)

In [336]:
# Convertir la columna 'fecha_reelevamiento' a tipo datetime sin la hora
df_final['fecha_reelevamiento'] = pd.to_datetime(df_final['fecha_reelevamiento'], format='%Y-%m-%d').dt.normalize()

# Convertir la columna 'Vencimiento' a tipo datetime sin la hora
df_final['Vencimiento'] = pd.to_datetime(df_final['Vencimiento'], format='%Y-%m-%d').dt.normalize()

# Convertir la columna 'SKU' a texto y quitar el '.0'
df_final['SKU'] = df_final['SKU'].astype(int)

In [337]:
df_final['fecha_reelevamiento'] = df_final['fecha_reelevamiento'].astype(str)
df_final['Vencimiento'] = df_final['Vencimiento'].astype(str)

In [338]:
df_final

,SKU,fecha_reelevamiento,Vencimiento,Ubicacion,TOTAL Bultos
0,25184,2023-05-22,2023-07-05,1,289.0
1,22639,2023-05-22,2023-07-09,1,12.0
2,24112,2023-05-22,2023-07-24,1,2.0
3,25232,2023-05-22,2023-07-28,1,129.0
4,20475,2023-05-22,2023-08-02,1,327.0
...,...,...,...,...,...
5752,24927,2023-11-17,2024-02-05,1,24.0
5753,24927,2023-11-17,2024-03-19,1,42.0
5754,21122,2023-11-17,2024-02-26,1,10.0
5756,9375,2023-11-17,2024-07-21,3,4680.0


In [339]:
# Abre un archivo para registrar los errores
with open("errores_log.txt", "w") as error_log:

    for index, fila in df_final.iterrows():
        try:
            # Preparar tu consulta SQL y los valores
            query = "INSERT INTO vencimientos (Codigo, fecha_relevamiento, fecha_vencimiento, codUbicacion, bultos) VALUES (%s, %s, %s, %s, %s)"
            values = (fila['SKU'], fila['fecha_reelevamiento'], fila['Vencimiento'], fila['Ubicacion'], fila['TOTAL Bultos'])

            # Ejecutar la consulta
            cursor.execute(query, values)
            db.commit()

        except mysql.connector.IntegrityError as e:
            print(f"Error al insertar la fila {index}: {e}")
            
            # Escribir el error y el código problemático en el archivo de log
            error_log.write(f"Fila {index}, Código: {fila['SKU']}, Error: {e}\n")
            
            # Continuar con la siguiente fila
            continue

Error al insertar la fila 0: 1062 (23000): Duplicate entry '25184-2023-05-22-1-2023-07-05' for key 'vencimientos.PRIMARY'
Error al insertar la fila 1: 1062 (23000): Duplicate entry '22639-2023-05-22-1-2023-07-09' for key 'vencimientos.PRIMARY'
Error al insertar la fila 2: 1062 (23000): Duplicate entry '24112-2023-05-22-1-2023-07-24' for key 'vencimientos.PRIMARY'
Error al insertar la fila 3: 1062 (23000): Duplicate entry '25232-2023-05-22-1-2023-07-28' for key 'vencimientos.PRIMARY'
Error al insertar la fila 4: 1062 (23000): Duplicate entry '20475-2023-05-22-1-2023-08-02' for key 'vencimientos.PRIMARY'
Error al insertar la fila 5: 1062 (23000): Duplicate entry '25233-2023-05-22-1-2023-08-05' for key 'vencimientos.PRIMARY'
Error al insertar la fila 6: 1062 (23000): Duplicate entry '25140-2023-05-22-1-2023-08-07' for key 'vencimientos.PRIMARY'
Error al insertar la fila 7: 1062 (23000): Duplicate entry '25144-2023-05-22-1-2023-08-09' for key 'vencimientos.PRIMARY'
Error al insertar la fil